There will be a lot decisions made regarding features based on the analysis performed earlier. It can accessed in analysis.ipynb file.

In [1]:
import pandas as pd
import numpy as np
import datetime 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
creditCardData = pd.read_csv("Data/creditcard.csv")

In [3]:
# Feature engineering

# As the time provided is in seconds we can use it as seconds since epoch as we won't care about years
def convert_totime(seconds):
    return datetime.datetime.fromtimestamp(seconds);

creditCardData['datetime'] = creditCardData.Time.apply(convert_totime)
creditCardData['hour of the day'] = creditCardData.datetime + pd.Timedelta("7 hour")
creditCardData['hour of the day'] = creditCardData['hour of the day'].dt.hour
creditCardData['isNight'] = (creditCardData['hour of the day'] >= 1) & (creditCardData['hour of the day'] <= 7)

#dropping insignificant rows
creditCardData = creditCardData.drop(['V22','V23','V25','V26','V13','V15', 'datetime'], axis = 1)

In [4]:
creditCardData['isNight'] = creditCardData['isNight'].map({False:0, True:1})
train, test  = train_test_split(creditCardData, test_size=0.33, random_state=42)

In [18]:
model = RandomForestClassifier(n_estimators = 500, max_depth = 8, max_features = 'sqrt')
train_y = train.Class
train_x = train.drop('Class', axis = 1)
model.fit(train_x, train_y)
test_y = test.Class
test_x = test.drop('Class', axis = 1)
preds = model.predict(test_x)
preds_prob = model.predict_proba(test_x)

In [19]:
print 'Accuracy:', accuracy_score(test_y, preds)
print 'AUC_ROC:', roc_auc_score(test_y, [x[1] for x in preds_prob])

Accuracy: 0.999595688765
AUC_ROC: 0.973519621349


Its a good AUC_ROC. Lets see if we can make it even better with some parameter tuning

In [13]:
from sklearn.model_selection import GridSearchCV

In [16]:
param_grid = {'max_depth': range(7,9), 'n_estimators': (100, 500, 1000)}

grid = GridSearchCV(RandomForestClassifier(max_features='sqrt'),
                    param_grid=param_grid, cv=5, scoring='roc_auc')
grid.fit(train_x, train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': (100, 500, 1000), 'max_depth': [7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

Ask in the comments about whether if u did the grid search with all the values does is consitutes as information leakage

In [17]:
grid.grid_scores_

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.96946, std: 0.01332, params: {'n_estimators': 100, 'max_depth': 7},
 mean: 0.97093, std: 0.01570, params: {'n_estimators': 500, 'max_depth': 7},
 mean: 0.97184, std: 0.01460, params: {'n_estimators': 1000, 'max_depth': 7},
 mean: 0.97223, std: 0.01573, params: {'n_estimators': 100, 'max_depth': 8},
 mean: 0.97261, std: 0.01450, params: {'n_estimators': 500, 'max_depth': 8},
 mean: 0.97253, std: 0.01542, params: {'n_estimators': 1000, 'max_depth': 8}]